In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

In [ ]:
from GPy.kern import RBF

class RBFDerivative(RBF):
    
    def Kxx(self, X, X2=None):
        return super(RBFDerivative, self).K(X,X2)

    def K(self,X,X2):
        k = super(RBFDerivative, self).K(X,X2)
        if X2 is None:
            diff = np.zeros((X.shape[0],X.shape[0]))
            for i in range(X.shape[0]):
                for j in range(X.shape[0]):
                    diff[i,j] = X[i,0] - X[j,0]
                    #diff[i,j] = -X[i,0] + X[j,0]

            return (1./(self.lengthscale[0]))*(1-(1./(self.lengthscale[0]))*(diff**2))*k
            #return k * (-1./(self.lengthscale[0]))*diff
        else:
            #X2 is derivative obs

            diff = np.zeros((X.shape[0],X2.shape[0]))
            for i in range(X.shape[0]):
                for j in range(X2.shape[0]):
                    diff[i,j] = X[i,0] - X2[j,0]
                    #diff[i,j] = -X[i,0] + X[j,0]

            return k * (1./(self.lengthscale[0]))*diff
            #return (1./(self.lengthscale[0]))*(1-(1./(self.lengthscale[0]))*(diff**2))*k


In [ ]:
x = np.linspace(1e-1,1)[:,None]

k = RBFDerivative(1, variance=.1)
k

In [ ]:
plt.imshow(k.K(x))

In [ ]:
plt.imshow(k.K(x,x))

In [ ]:
s = scipy.stats.multivariate_normal.rvs(np.zeros(50), k.K(x), size=10)

plt.plot(x[:,0],s.T);

In [ ]:
t = 100
nrep = 3

x2 = np.zeros((t*nrep, 2))
x2[:,0] = np.tile(np.linspace(1e-9,1,t), nrep)
x2[:,1] = np.repeat(range(nrep), t)

# k2 = GPy.kern.Hierarchical([GPy.kern.Integral(1, variances=1.), GPy.kern.RBF(1)])
# k2 = GPy.kern.Hierarchical([GPy.kern.Integral(1, variances=1.), GPy.kern.Integral(1, variances=.1)])
k2 = GPy.kern.Hierarchical([RBFDerivative(1), RBFDerivative(1, variance=.1, lengthscale=.25)])

In [ ]:
x2.shape

In [ ]:
plt.imshow(k2.K(x2))

In [ ]:
plt.imshow(k2.K(x2, x2))

In [ ]:
s = scipy.stats.multivariate_normal.rvs(np.zeros(t*nrep), k2.K(x2))

plt.plot(s.reshape(nrep,t).T);

In [ ]:
n = x2.shape[0]

# rep = np.array([1,0,0,0,0]*20 + [0,1,0,0,0]*20 + [0,0,1,0,0]*20 + [0,0,0,1,0]*20 + [0,0,0,0,1]*20).reshape((n,5))
rep = np.array([1,0,0,0,0]*t + [0,1,0,0,0]*t + [0,0,1,0,0]*t).reshape((n,5))

cov = np.zeros((n*2, n*2))

cov[:n,:n] = k2.K(x2)
cov[:n,n:] = k2.K(x2, x2).T
cov[n:,:n] = k2.K(x2, x2)
cov[n:,n:] = k2.rbf.Kxx(x2) + k2.rbf_1.Kxx(x2)*np.dot(rep,rep.T) + np.eye(t*nrep)*1e-3

plt.imshow(cov)


In [ ]:
s = scipy.stats.multivariate_normal.rvs(np.zeros(n*2), cov).reshape(nrep*2,t).T

dy, y = np.array_split(s, 2, 1)

# plt.plot(s.reshape(10,20).T);
plt.figure(figsize=(8,4))
# plt.subplot(121)
plt.plot(dy);
plt.plot([0,t],[0,0],'k',lw=3)
# plt.subplot(122)
plt.twinx()
ax = plt.gca()
plt.gca().set_color_cycle(None)
plt.plot(y, '--')

In [ ]:
def gompertz(t,m,A,l):
    return A * np.exp(-np.exp(m*np.e/A*(l-t) + 1))

In [ ]:
def add_subplot_axes(ax,rect,axisbg='w'):
    fig = plt.gcf()
    box = ax.get_position()
    width = box.width
    height = box.height
    inax_position  = ax.transAxes.transform(rect[0:2])
    transFigure = fig.transFigure.inverted()
    infig_position = transFigure.transform(inax_position)    
    x = infig_position[0]
    y = infig_position[1]
    width *= rect[2]
    height *= rect[3]  # <= Typo was here
    subax = fig.add_axes([x,y,width,height],axisbg=axisbg)
    x_labelsize = subax.get_xticklabels()[0].get_size()
    y_labelsize = subax.get_yticklabels()[0].get_size()
    x_labelsize *= rect[2]**0.5
    y_labelsize *= rect[3]**0.5
    subax.xaxis.set_tick_params(labelsize=x_labelsize)
    subax.yaxis.set_tick_params(labelsize=y_labelsize)
    return subax


In [ ]:
# kbase = GPy.kern.RBF(1,name='base')
# kbio = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1)])
# ktech = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1), GPy.kern.RBF(1,name='tech', variance=.05)])

In [ ]:
nbatch = 3
nrep = 3
ntot = nbatch * nrep

x = np.zeros((50*ntot,3))
x[:,0] = np.tile(np.linspace(0,2), ntot)
x[:,1] = np.repeat(np.arange(nbatch), nrep*50)
x[:,2] = np.repeat(np.arange(nbatch*nrep), 50)

In [ ]:
baseVariance, batchVariance, repVariance = .01, .01, .005

kbase = GPy.kern.RBF(1,name='base',variance=baseVariance)
kbatch = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='batch', variance=batchVariance), index_dim=-2);
krep = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='replicate', variance=repVariance))

In [ ]:
plt.imshow(kbatch.K(x))

plt.figure()
plt.imshow(krep.K(x))

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(221)
plt.imshow(kbase.K(x))

plt.subplot(222)
plt.imshow(kbatch.K(x) + kbase.K(x))

plt.subplot(223)
plt.imshow(krep.K(x) + kbase.K(x))

plt.subplot(224)
plt.imshow(krep.K(x) + kbatch.K(x) + kbase.K(x))

plt.savefig("figures/simulated-kernel.pdf",bbox_inches='tight')

In [ ]:
def generateSample(mu, cov, nugget, length=50):
    noise = np.eye(mu.shape[0])*nugget
    
    return scipy.stats.multivariate_normal.rvs(mu,cov+noise).reshape((mu.shape[0]/length,length)).T

In [ ]:
cov = np.zeros((x.shape[0]*4, x.shape[0]*4))

# cov = kbase.K(np.tile(x.T,4).T) #+ batchVariance + repVariance

print cov.shape

# cov[:,:x.shape[0]] = np.tile(kbase.K(x), 4).T
# cov[:x.shape[0],:] = np.tile(kbase.K(x), 4)
# cov[:x.shape[0],:] = kbase.K(x)
cov[x.shape[0]:2*x.shape[0],x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance
cov[x.shape[0]:2*x.shape[0],3*x.shape[0]:] += kbatch.K(x) #- batchVariance
cov[3*x.shape[0]:,x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance

cov[2*x.shape[0]:3*x.shape[0],2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance
cov[2*x.shape[0]:3*x.shape[0],3*x.shape[0]:] += krep.K(x) #- repVariance
cov[3*x.shape[0]:,2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance

cov[3*x.shape[0]:,3*x.shape[0]:] += kbatch.K(x) + krep.K(x) # - repVariance-  batchVariance

# equal variance
# cov[range(cov.shape[0]),range(cov.shape[0])] = np.diag(cov).max()

plt.imshow(cov)

In [ ]:
np.random.seed(1)

sigma = .001
f = gompertz(x[:,0], 2, 1, .4)
s = generateSample(np.tile(f, 4), cov, sigma)

In [ ]:
plt.figure(figsize=(6,6))
for i in range(4):
    ax = plt.subplot(2,2,i+1)
    
    plt.title('$M_%d$'%[0,2,1,3][i])
    
    if i > 1:
        plt.xlabel("time (AU)",)
    else:
        plt.xticks([0,20,40],['']*3)
        
    if i % 2 == 0:
        plt.ylabel("OD (AU)")
    else:
        plt.yticks(np.arange(-.2,1.4,.2), ['']*8)
    plt.plot(x[:50,0],f[:50],c='k', lw=3)
    
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(x[:50,0],s[:,ntot*i:ntot*(i+1)][:,j],color='C%d'%k,alpha=.6);
            
    plt.ylim(-.24, 1.29)
    
    subpos = [.45,.05,.5,.4]
    a = add_subplot_axes(ax,subpos)
    #n, bins, patches = plt.hist(x[:,0], 400, normed=1)
    plt.plot([x[:,0].min(), x[:,0].max()], [0,0], c='k', lw=3)
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(x[:50,0],s[:,ntot*i:ntot*(i+1)][:,j]-f[:50],color='C%d'%k, alpha=.5);
    plt.xticks([])
    plt.yticks([])
            
#plt.tight_layout()
plt.savefig("figures/simulated-data.pdf", bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,10))
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.plot([0, 50], [0,0], c='k', lw=3)
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(s[:,ntot*i:ntot*(i+1)][:,j]-f[:50],color='C%d'%k);

In [ ]:
plt.figure(figsize=(10,10))

sigma = .001

plt.subplot(221)
cov = kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

plt.subplot(222)
cov = kbatch.K(x) + kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

plt.subplot(223)
cov = krep.K(x) + kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

plt.subplot(224)
cov = kbatch.K(x) + krep.K(x) + kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

In [ ]:
sigma = .001
# noise = np.eye(50*ntot)*sigma

f = gompertz(x[:,0]+1, 2, 1, .4)

np.random.seed(1)

plt.figure(figsize=(12,4))
for i,kern in enumerate([None, kbio, ktech]):

    mu = np.zeros(x.shape[0])
    
    if not kern is None:
        cov = kern.K(x)
        f += scipy.stats.multivariate_normal.rvs(mu,cov)
        
    extraVariance = sum([0, bioVariance, techVariace][i+1:])
    noise = np.eye(50*ntot)*(sigma+extraVariance)
    
    s = scipy.stats.multivariate_normal.rvs(f,noise).reshape((ntot,50)).T
    
    plt.subplot(1,3,i+1)
    
#     if i > 0:
#         for j,z in enumerate(x[::50,i]):
#             k = np.unique(x[:,i]).tolist().index(z)
#             plt.plot(s[:,j],color='C%d'%k);
#     else:
#         plt.plot(s,c='k');
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(s[:,j],color='C%d'%k);
            
    plt.ylim(-.25,1.3)
        
plt.savefig("figures/simulated-data.pdf",bbox_inches='tight')